In [32]:
import random
import optuna

import mlflow
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor

In [ ]:
filename = 'data/2018-01-01-2022-01-01.bin'

cols_to_drop = ['subba-name', 'parent', 'parent-name', 'value-units']

dtypes = {'period': 'datetime64',
          'subba': 'category',
          'timezone': 'category',
          'value': 'uint64'}

df = pd.read_pickle(filename).drop(columns=cols_to_drop)
df = df.drop_duplicates().dropna().reset_index(drop=True)
df = df.astype(dtypes)

df = df[df['subba'] == 'ZONJ']
print('df shape:', df.shape)
print(df.head(1))
print(df.tail(1))

In [ ]:
filt = df.value == 0
to_drop = df[filt].index
df = df.drop(index=to_drop).reset_index(drop=True)

In [ ]:
df.sort_values(by='period').plot(x='period', y='value', figsize=(10, 5));

In [ ]:
# Compute the IQR for the "value" column
Q1 = df['value'].quantile(0.25)
Q3 = df['value'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for the acceptable range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove rows where the value in the "value" column is outside the bounds
df_filtered = df[(df['value'] >= lower_bound) & (df['value'] <= upper_bound)]

print(df_filtered)

# Plotting
df_filtered.plot(x='period', y='value', kind='line')

plt.xlabel('Period')
plt.ylabel('Value')
plt.title('Value over Time')
plt.show()

In [ ]:
# Sort df by date in ascending order
df_sorted = df.sort_values(by=['period']).reset_index(drop=True)

In [ ]:
def extract_features(df):
    # Separate date features
    df['year'] = df['period'].dt.year.astype('uint16')
    df['month'] = df['period'].dt.month.astype('uint16')
    df['day'] = df['period'].dt.day.astype('uint16')

    # Extract the day of the week
    df['day_of_week'] = df['period'].dt.day_name()

    # Extract quarterly and weekly information to capture seasonality
    df['quarter'] = df['period'].dt.quarter.astype('uint8')
    df['week_of_year'] = df['period'].dt.isocalendar().week.astype('uint16')

    # Mark the weekends
    df['is_weekend'] = (df['period'].dt.weekday >= 5).astype('uint8')
    
    window_size = 7 # 7-day window

    # Create rolling mean feature
    df['rolling_mean_7'] = df['value'].rolling(window=window_size).mean()

    # Create rolling standard deviation feature
    df['rolling_std_7'] = df['value'].rolling(window=window_size).std()
    
    # Remove NaNs introduced by the rolling mean and std
    df = df.dropna().copy().reset_index(drop=True)

    # Create a random feature as a threshold for later feature filtering
    random_feature = [random.random() for _ in range(df.shape[0])]
    df['random_feature'] = random_feature
    return df

df_newfeat = extract_features(df_sorted)

In [ ]:
def transform_to_supervised(df):
    print(df.columns.tolist())
    df.loc[:, 'lag'] = df['value'].shift()
    df.dropna(inplace=True)
    df.lag = df.lag.astype('uint64')
    df = df.drop(columns=['period'])
    return df

def encode_categorical(df):
    cols_to_enc = ['timezone', 'day_of_week']

    dummies = pd.get_dummies(df[cols_to_enc])
    df_encoded = pd.concat((dummies, df.drop(columns=cols_to_enc)), axis=1)
    return df_encoded

In [ ]:
# Prepare the dataset for feeding it into the model
df_transformed = transform_to_supervised(df_newfeat)
df_transformed
df_encoded = encode_categorical(df_transformed).drop(columns=['subba'])
print(df_encoded.shape)
df_encoded.sample(5)

In [ ]:
X = df_encoded.drop(columns=['value'])
y = df_encoded.value

In [ ]:
n_splits = 9
tscv = TimeSeriesSplit(n_splits=n_splits)

In [ ]:
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

In [ ]:
def train(model, X, y):
    mae_train_hist = []
    mae_test_hist = []
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        mae_train = mean_absolute_error(y_train, y_pred_train)
        mae_test = mean_absolute_error(y_test, y_pred_test)
        
        mae_train_hist.append(mae_train)
        mae_test_hist.append(mae_test)

    mae_train_avg = sum(mae_train_hist) / len(mae_train_hist)
    mae_test_avg = sum(mae_test_hist) / len(mae_test_hist)
    return (mae_train_avg, mae_test_avg)

mae_train_avg, mae_test_avg = train(model, X, y)
print(f'Average mae on {n_splits} splits:\nTrain: {mae_train_avg} | Test: {mae_test_avg}')

In [ ]:
imp = model.feature_importances_
cols = X.columns.tolist().copy()
feat_imps = pd.DataFrame((cols, imp)).T
feat_imps.columns = ['feature', 'importance']
feat_imps.sort_values(by=['importance'], ascending=False).reset_index(drop=True)

In [ ]:
threshold = feat_imps[feat_imps['feature'] == 'random_feature']['importance']
redundant_col = feat_imps[feat_imps['importance'] < threshold.iloc[0]]
to_drop = redundant_col['feature'].tolist() + ['random_feature']
X_new = X.drop(columns=to_drop)

In [ ]:
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
mae_test_avg = train(model, X_new, y)

In [ ]:
cols_to_drop = to_drop + ['value']
cols_to_drop

In [ ]:
test_row = df_encoded.iloc[-1:]
test_row = test_row.drop(columns=cols_to_drop)
test_row

In [ ]:
model.predict(test_row)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 2, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'alpha': trial.suggest_float("alpha", 1e-5, 10, log=True),  
        'gamma': trial.suggest_float("gamma", 1e-5, 5, log=True),
        'lambda': trial.suggest_float('lambda', 1e-5, 10.0, log=True),
        'early_stopping_rounds': 50
    }

    model = XGBRegressor(**params)
    mae_train_avg, mae_test_avg = train(model, X_new, y)
    return mae_test_avg

In [ ]:
optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction='minimize')

In [ ]:
# Optimize the study, the objective function is passed in as the first argument
study.optimize(objective, n_trials=100)

In [ ]:
# Results
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
best_params = trial.params
model = XGBRegressor(**best_params)
train(model, X_new, y)

In [ ]:
model.predict(test_row)